In [1]:
import tensorflow as tf
from collections import defaultdict
import gzip
import cv2 as cv
import os
import numpy as np
import random

/home/sudhakaran/Desktop/3d_recognition_thesis/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sudhakaran/Desktop/3d_recognition_thesis/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sudhakaran/Desktop/3d_recognition_thesis/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)]

In [10]:
def extract_data(self):
    train = []
    test = []
    train_labels = []
    test_labels = []
    label_names = []
    IMG_SIZE = 128
    IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
    save_model_path = '../Saved_Models/custom_MobileNetV2.h5'
    
# -----------Loading model ------------------

    model = tf.keras.models.load_model(save_model_path)
    model.trainable = False
    model.summary()

    print("\n..........loading dataset from disk..........\n")

    for label in sorted(os.listdir(data_path)):
        label_path = os.path.join(data_path, label)

        for file in (os.listdir(label_path)):
            file_path = os.path.join(label_path, file)

            if str(file) == "train":
                for instance in sorted(os.listdir(file_path)):
                    instance_path = os.path.join(file_path, instance)
                    train_views = []

                    for img in sorted(os.listdir(instance_path)):
                        img_path = os.path.join(instance_path, img)
                        image = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
                        re_image = cv.resize(image, (IMG_SIZE, IMG_SIZE))
                        train_views.append(re_image)
                    image1 = cv.merge(train_views)
                    train.append(image1)
                    train_labels.append(label)
    #                 cv.imshow("image", image)
    #                 cv.waitKey(0)
    #                 cv.destroyAllWindows()
    #                 cv.imwrite("./2.jpg", image1)
    #                 break

            elif str(file) == "test":
                for instance in sorted(os.listdir(file_path)):
                    instance_path = os.path.join(file_path, instance)
                    test_views = []

                    for img in sorted(os.listdir(instance_path)):
                        img_path = os.path.join(instance_path, img)
                        image = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
                        re_image = cv.resize(image, (IMG_SIZE, IMG_SIZE))
                        test_views.append(re_image)
                    image2 = cv.merge(test_views)
                    test.append(image2)
                    test_labels.append(label)

    train = np.array(train, dtype="float") / 255.0
    train_features = model.predict(train, batch_size=5)
    print(train_features.shape)
#     train = train.reshape(train.shape[0], 128, 128, 3)
    train_labels = np.array(train_labels)

    test = np.array(test, dtype="float") / 255.0
    test_features = model.predict(test, batch_size=5)
    print(test_features.shape)
#     test = test.reshape(test.shape[0], 128, 128, 3)
    test_labels = np.array(test_labels)

    label_names = np.unique(train_labels)

    return train_features, train_labels, test_features, test_labels, label_names

In [11]:
if __name__ == "__main__":
    
    data_path = '../ModelNet40/image_dataset'
    
    np_train_path = '../ModelNet40/extracted_features/custom_MobileNetV2/train.npy.gz'
    np_test_path = '../ModelNet40/extracted_features/custom_MobileNetV2/test.npy.gz'
    np_train_label_path = '../ModelNet40/extracted_features/custom_MobileNetV2/train_labels.npy'
    np_test_label_path = '../ModelNet40/extracted_features/custom_MobileNetV2/test_labels.npy'
    np_label_names_path = '../ModelNet40/extracted_features/custom_MobileNetV2/label_names.npy'
       
    train_data, train_labels, test_data, test_labels, label_names = extract_data(data_path)

    os.makedirs(os.path.dirname(np_train_path), exist_ok=True)

    with gzip.GzipFile(np_train_path, "w") as f:
        np.save(f, train_data)
    with gzip.GzipFile(np_test_path, "w") as f:
        np.save(f, test_data)

    np.save(np_train_label_path, train_labels)
    np.save(np_test_label_path, test_labels)
    np.save(np_label_names_path, label_names)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 1280)              2257984   
Total params: 4,481,856
Trainable params: 2,223,872
Non-trainable params: 2,257,984
_________________________________________________________________

..........loading dataset from disk..........

(9843, 1280)
(2468, 1280)
